<a href="https://colab.research.google.com/github/Thayer-ENGS108/Assignment_6_Fall2022/blob/main/assignment_6_Fall2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ENGS 108 Fall 2022 Assignment 6**

*Due November 11, 2022 at 11:59PM on Github*

**Instructors:** George Cybenko

**TAs:** Chase Yakaboski and Clement Nyanhongo


---

## **Rules and Requirements**


1.   You are only allowed to use Python packages that are explicity imported in 
the assignment notebook or are standard (bultin) python libraries like random, os, sys, etc, (Standard Bultin Python libraries will have a Python.org documentation). For this assignment you may use:
  *   [numpy](https://numpy.org/doc/stable/)
  *   [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
  *   [scikit-learn](https://scikit-learn.org/stable/)
  *   [matplotlib](https://matplotlib.org/)
  *   [tensorflow](https://www.tensorflow.org/)

2.   All code must be fit into the designated code or text blocks in the assignment notebook. They are indentified by a **TODO** qualifier.

3. For analytical questions that don't require code, type your answer cleanly in Markdown. For help, see the [Google Colab Markdown Guide](https://colab.research.google.com/notebooks/markdown_guide.ipynb).

---


In [4]:
!pip install --user -U nltk

In [2]:
''' Import Statements '''
import os
import random
import numpy as np
import pandas as pd
import csv
import tqdm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/t/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Creating a Trump Tweet Generator!!!
Finally it's happened. I am allowed to make my dream assignment. Trust me it's the best assignment. People tell me all the time how they wish they could have had an assignment as great as this. You will see why! But first some housekeeping... 

# Data Loading
In this assignment we will be using a repository of Donald Trump tweets scrapped from Twitter through June 2020 from [Kaggle](https://www.kaggle.com/datasets/austinreese/trump-tweets) and will use the following code blocks to download the dataset directly to your Google Drive.

## Creating a Kaggle API Token
First we will need to download an API token from Kaggle in order to download the dataset, so our first step is to create a Kaggle account if you don't already have one. (You should have done this in Assignment 4 in case this looks familiar. 
1. Create a Kaggle account by following the sign up instructions [here](https://www.kaggle.com/).
2. Log into your Kaggle account and click your account icon on the upper righthand side. 
3. Then select **Account** from the dropdown/sidebar menu.
4. Scroll down to the **API** section and select **Create New API Token**.
5. This will download a JSON file called kaggle.json to your Downloads folder on your computer.
6. Now run the following code block and when the **Browse** button appears, click it and select that kaggle.json file. 

# Run this code block after creating a Kaggle API token as instructed above.
! pip install -q kaggle


#Will ask you to upload kaggle.json file and remove any old ones.
if os.path.exists('kaggle.json'):
  os.remove('kaggle.json')
#files.upload()

# Will create the appropriate directory structure
if not os.path.exists('/root/.kaggle'):
  ! mkdir ~/.kaggle 
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
# Also we are going to make a directory called result
if not os.path.exists('/content/results'):
  ! mkdir /content/result

## Downloading the Dataset

7. Now we have downloaded our Kaggle credentials we can now download the Trump Tweets Dataset (or any other Kaggle dataset for that matter) directly into our Google Drive.

! kaggle datasets download austinreese/trump-tweets
# Will check to see if the yoga postures zip file has been unzipped and will unzip the file if not.
! unzip trump-tweets.zip

!y


## Loading the Dataset using Pandas
Now let's inspect the trump tweets dataset and see what we have to work with... Brace yourselves.

In [3]:
# Let's load in the two files that we inflated from the Kaggle download. Both realdonaldtrump.csv and trumptweets.csv are the same.
real_donald_trump_df = pd.read_csv('realdonaldtrump.csv')

In [4]:
real_donald_trump_df.head()

,id,link,content,date,retweets,favorites,mentions,hashtags
0,1698308935,https://twitter.com/realDonaldTrump/status/169...,Be sure to tune in and watch Donald Trump on L...,2009-05-04 13:54:25,510,917,NaN,NaN
1,1701461182,https://twitter.com/realDonaldTrump/status/170...,Donald Trump will be appearing on The View tom...,2009-05-04 20:00:10,34,267,NaN,NaN
2,1737479987,https://twitter.com/realDonaldTrump/status/173...,Donald Trump reads Top Ten Financial Tips on L...,2009-05-08 08:38:08,13,19,NaN,NaN
3,1741160716,https://twitter.com/realDonaldTrump/status/174...,New Blog Post: Celebrity Apprentice Finale and...,2009-05-08 15:40:15,11,26,NaN,NaN
4,1773561338,https://twitter.com/realDonaldTrump/status/177...,"""My persona will never be that of a wallflower...",2009-05-12 09:07:28,1375,1945,NaN,NaN


## Problem 1: Pre-Processing the Tweets
As you may have noticed from the dataframe we just loaded, there are some special characters we need to handle. If we are making a tweet or sentence generator, we don't want to mess with special characters like commas or colons or really even captialization. So in the following section you are going to preprocess the data and strip these special characters out.

### Task 1: The Preprocess function
In the following code block complete the preprocess function that will strip or substitute out various special characters or sequences of characters that may not be ideal when training a sentence generator. We will use the built-in re python library to do a number of substitutions, and I have given you a skeleton below, see if you can complete it.

In [5]:
import re

REGEX_SUBS = {
  r'\\': ' ',
  r'\n': ' ',
  r'&': '',
  r'RT ': '',
  r'~': '',
  r'#': '',
  r'!+': '',
  r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)': 'link',
  r'[*]': '',
  r'[@]\w+': 'user',
  r'[:|;]': '',
  r'[\\x]\W+|\d': '',
  r'[\\x]\W+|\d': '',
  r'[\\x]\w+': '',
  r'    ': ' ',
  r'   ': ' ',
  r'  ': ' ',
  r'-': '',
  r",": '',
  r'"': '',
  r"'": '',

}

def preprocess(text):
  # TODO: Complete the function and using the provided regular expression substitutions.
    text = str(text)
    for key, value in REGEX_SUBS.items():
        text = re.sub(key, value, text).lower()
        
        #TODO: Do something, maybe look at the re.sub command.
    
    
    
  #TODO: Also make everything lowercase
    return text

### Task 2: Preprocess the dataset.
Now that we have our preprocess function, let's preprocess all the tweets.

In [6]:
rdt = np.array(real_donald_trump_df)[:,2]
rdt_proc = []
for i in rdt:
    rdt_proc.append(preprocess(i))
#TODO: Run all the Dataframe content through your preprocess function.

rdt = rdt_proc

### Task 3: Tokenize the Data
The next step in every Natural Language Processing task is to tokenize the data, i.e., seperate our words, special characters, etc. into separate uniquie tockens. We will be using the [Natural Language Tool Kit](https://www.nltk.org/index.html) in python to accomplish this. Study the nltk API docs and see if you can tokenize our data. 

In [7]:
from nltk.tokenize import word_tokenize
preproc = []
toke = []
for i in rdt:
    i = i.replace("/", "")
    i = i.replace(".","")
    i = i.replace(")","")
    i = i.replace("(","")
    i = i.replace("  "," ")
    i = i.replace("'\'", "")
    i = i.replace('’',"")
    i = i.replace("-", "")
    i = i.replace('–', "")
    i = i.replace('‘', "")
    i = i.replace("@","")
    i = i.replace("'”'","")
#     i = i.replace("''","")
    toke.append(word_tokenize(i))
    preproc.append(i+"\n")
    
# TODO: Tokenize each preprocessed tweet


In [8]:
preproc[0]

'be sure to tune in and watch donald trump on late night with david letterman as he presents the top ten list tonight\n'

### Task 4: Build a Vocabulary
Now that we have our data tokenized, let's build a vocabulary including beginning and ending of sentence tokens, i.e., \<s>, \</s> for example. At this point let's also add in these beginning and end tokens into each of our data instances. 

In [9]:
toke

[['be',
  'sure',
  'to',
  'tune',
  'in',
  'and',
  'watch',
  'donald',
  'trump',
  'on',
  'late',
  'night',
  'with',
  'david',
  'letterman',
  'as',
  'he',
  'presents',
  'the',
  'top',
  'ten',
  'list',
  'tonight'],
 ['donald',
  'trump',
  'will',
  'be',
  'appearing',
  'on',
  'the',
  'view',
  'tomorrow',
  'morning',
  'to',
  'discuss',
  'celebrity',
  'apprentice',
  'and',
  'his',
  'new',
  'book',
  'think',
  'like',
  'a',
  'champion'],
 ['donald',
  'trump',
  'reads',
  'top',
  'ten',
  'financial',
  'tips',
  'on',
  'late',
  'show',
  'with',
  'david',
  'letterman',
  'link',
  'very',
  'funny'],
 ['new',
  'blog',
  'post',
  'celebrity',
  'apprentice',
  'finale',
  'and',
  'lessons',
  'learned',
  'along',
  'the',
  'way',
  'link'],
 ['my',
  'persona',
  'will',
  'never',
  'be',
  'that',
  'of',
  'a',
  'wallflower',
  'id',
  'rather',
  'build',
  'walls',
  'than',
  'cling',
  'to',
  'them',
  'donald',
  'j',
  'trump'],
 [

In [13]:
from collections import Counter
start = "<s>"
end = "</s>"
marked = []
for i in toke:
    new = i
    new.insert(0,start)
    new.append(end)
    marked.append(new)
flat = [i for j in marked for i in j]
nw = len(set(flat))
count = dict(Counter(flat))
voc = {start:0, end:1}
for val,key in enumerate(set(flat).symmetric_difference({start,end})):
    voc[key] = val + 2
voci = {val:key for key,val in voc.items()}
dtraj = list(map(lambda x:voc[x], flat))
countmat = np.zeros((nw,nw))
for i in range(len(dtraj) - 1):
    countmat[(dtraj[i+1],dtraj[i])] += 1 #column stochastic count matrix for 1 step transitions
tmat = (countmat/countmat.sum(0)).T #normed row transition matrix




# Problem 1: N-gram Language Model
In this problem, you will be building a couple n-gram language modeling and see how well just taking pure frequency counts and building a conditional probability distrbution will work.

## Task 1: A 2-gram (Bigram) Model
Recall that our goal in building a language model is to represent the conditional probability $P(w_i | w_{i-1})$ for pairs of words $w_i$ and $w_j$.

### Part A: Frequencies
Go through the encoded Trump tweets and calculate the frequencies of all words as well as all pair of words that appear next to each other in the corpus. 

In [15]:
##done in the above cell
count

{'<s>': 43352,
 'be': 6913,
 'sure': 311,
 'to': 19751,
 'tune': 112,
 'in': 11848,
 'and': 15577,
 'watch': 708,
 'donald': 1829,
 'trump': 5950,
 'on': 7984,
 'late': 125,
 'night': 683,
 'with': 5116,
 'david': 132,
 'letterman': 84,
 'as': 2444,
 'he': 3849,
 'presents': 12,
 'the': 34407,
 'top': 305,
 'ten': 31,
 'list': 88,
 'tonight': 752,
 '</s>': 43352,
 'will': 6633,
 'appearing': 14,
 'view': 83,
 'tomorrow': 453,
 'morning': 364,
 'discuss': 66,
 'celebrity': 284,
 'apprentice': 495,
 'his': 2086,
 'new': 1951,
 'book': 413,
 'think': 1029,
 'like': 1723,
 'a': 14904,
 'champion': 178,
 'reads': 10,
 'financial': 76,
 'tips': 11,
 'show': 836,
 'link': 9437,
 'very': 2499,
 'funny': 74,
 'blog': 19,
 'post': 191,
 'finale': 34,
 'lessons': 19,
 'learned': 40,
 'along': 159,
 'way': 880,
 'my': 3805,
 'persona': 7,
 'never': 1480,
 'that': 5835,
 'of': 13321,
 'wallflower': 1,
 'id': 137,
 'rather': 111,
 'build': 196,
 'walls': 23,
 'than': 1414,
 'cling': 1,
 'them': 1057

### Part B: Probabilities
Now from the counts above we will calculate an associated conditional probabilities.

In [16]:
##I made a count transition matrix and normalized it....
tmat

array([[0.00000000e+00, 2.30669865e-05, 4.61339731e-05, ...,
        0.00000000e+00, 2.30669865e-05, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

### Part C: Make an Bigram Generator Function
Now that we have our probability distrbution, let's make a generator function so that we can generate random Trump tweets using our bigram language model.

In [17]:
def get_next_word(state,tmat):
    dist = tmat[state]; idx = np.where(dist != 0)[0]
    return np.random.choice(idx.tolist(),1,dist[idx].tolist())[0]

def generate(start_text='', n=10, tmat=tmat, voc=voc, voci=voci):
    
    # Helper code to create the start text. 
    start_text = ['<s>'] + nltk.word_tokenize(preprocess(start_text))
    new = []
    word = voc[start_text[-1]]
    c = 0
    while (word != 1) and (c<=n):
        word = get_next_word(word, tmat)
        c +=1
        new.append(word)
    new = list(map(lambda x:voci[x],new))
    out = start_text + new
    return " ".join(out)

In [27]:
generate('I love')

'<s> i love how dishonest weekly apprentice hardly knew there plans health plan stay'

## **(Bonus)** Task 2: Make a Trigram Model
Using your code from Parts A-C, see if you can build a trigram (3-gram) model that might make tweets a little more logical. 

#TODO: Your code goes here.
countmat3 = np.zeros((nw,nw,nw))
for i in range(len(dtraj) - 2):
    countmat3[(dtraj[i+2],dtraj[i+1],dtraj[i])] += 1 #column stochastic count matrix for 1 step transitions
tmat3 = (countmat3/countmat3.sum(0)).T

def get_next_word3(state,tmat):
    dist = tmat[state]; idx = np.where(dist != 0)[0]
    return np.random.choice(idx.tolist(),1,dist[idx].tolist())[0]

def generate3(start_text='', n=10, tmat=tmat, voc=voc, voci=voci):
    
    # Helper code to create the start text. 
    start_text = ['<s>'] + nltk.word_tokenize(preprocess(start_text))
    new = []
    word1 = voc[start_text[-2]]
    word2 = voc[start_text[-1]]
    c = 0
    while (c<=n) and ((word2 and word1) != (0 or 1)):
        new = get_next_word((word1,word2), tmat)
        word1 = word2
        word2 = new
        c +=1
        new.append(word2)
    new = list(map(lambda x:voci[x],new))
    out = start_text + new
    return " ".join(out)

# Problem 2: Using a Transformer
In this section we are going to leverage a pretrained transformer, i.e., [GPT-2](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf), built by the [Hugging Faces Team](https://huggingface.co/gpt2?text=A+long+time+ago%2C). We will also be using their tokenizers because they have been optimized for the language generation task. You should be aware that behind the scenes, their model is using [PyTorch](https://pytorch.org/), the deep learning library built by Facebook, and is quickly becoming more popular than our beloved Tensorflow. 

## Task 1: Install and load the GPT2 Model

In [31]:
# TODO: Follow the link above and load the GPT2 model as well as the tokenizer.
# NOTE: Replace GPT2LMHeadModel with GPT2LMHeadModel (this is the model for finetuning use cases.)

from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

## Task 2: Formatting our Training Data
Upcoming we will be fine-tunning this GPT-2 model on our Trump Tweets, but in order to leverage some of the utility classes built by HuggingFaces, we want to take our preprocessed Trump tweets and place them in a flat text file.

In [30]:
#TODO: Take our preprocessed trump tweets and write them to a text file
with open('trumpdata.txt', 'w') as text_file:
    text_file.writelines(preproc)
  #TODO: Do something

## Task 4: Fine-tuning the Model
In the following sections, we will complete a couple functions that will allow us to fine-tune the GPT-2 model to our Trump Tweets. I am going to give you a couple of these helper functions, but leave you to write parts of the training function. See the following documentation from Hugging Faces to see the attributes for the [Trainer Class](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer).

In [32]:
from transformers import Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(file_path, tokenizer, block_size = 128):
    # Will load and tokenize the data
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    # Helper Function 
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator

def train(
    train_file_path,
    model,
    tokenizer,
    output_dir,
    overwrite_output_dir,
    num_train_epochs,
    save_steps
    ):
  
  #TODO: Use the helper functions above to load the dataset and data collector.
  global train_dataset

  train_dataset = load_dataset(train_file_path,tokenizer)
  data_collator = load_data_collator(tokenizer)

  # Don't mess with.
  tokenizer.save_pretrained(output_dir)
  model.save_pretrained(output_dir)

  # Don't mess with.
  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=8,
          num_train_epochs=num_train_epochs,
      )
  #TODO: Use the Trainer class with the necessary parameters to instaniate the trainer
  trainer = Trainer(model,training_args,data_collator, train_dataset, tokenizer,
      
  )
  trainer.train()
  #trainer.save()
  
  #TODO: Train and save the model using the train and save functions built into the Trainer class.

In [33]:
#TODO: Set necessary parameters, here are some defaults.
train_file_path = "./trumpdata.txt"
output_dir = 'result'
overwrite_output_dir = False
num_train_epochs = 1
save_steps = 500

In [64]:
#TODO: Use your train function to train the model. It takes about 30 minutes to train in colab.
train(train_file_path, model, tokenizer,output_dir, overwrite_output_dir,num_train_epochs, save_steps)

Loading features from cached file ./cached_lm_GPT2Tokenizer_128_trumpdata.txt [took 0.034 s]
tokenizer config file saved in result/tokenizer_config.json
Special tokens file saved in result/special_tokens_map.json
Configuration saved in result/config.json
Model weights saved in result/pytorch_model.bin
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 8800
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1100
  Number of trainable parameters = 124439808


Step,Training Loss
500,4.088500
1000,3.977300


Saving model checkpoint to result/checkpoint-500
Configuration saved in result/checkpoint-500/config.json
Model weights saved in result/checkpoint-500/pytorch_model.bin
Saving model checkpoint to result/checkpoint-1000
Configuration saved in result/checkpoint-1000/config.json
Model weights saved in result/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




AttributeError: 'Trainer' object has no attribute 'save'

 # The above generates an error because of an accidentally uncommented line,however, the results were saved to disk before the erroneous line :) ...I have since commened out the line

## Task 5: Creating a Tweet Generator
Now that we have our trained model, it's time to generate some Tweets. Since we should have saved our model and tokenizer to an output directory, I've already made some helper functions to load those in. We will focus on our *generate_text* function. The function will take as input some start text like "I am" or "My country is", etc., as well as a max_length parameter which tells the model how much text to generate. Let's Go!

In [34]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model

def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(sequence, max_length):
    #TODO: Load in the finetuned model and tokenizer
    model = load_model("./result")
    tokenizer = load_tokenizer("./result")

    # Encode our passed sequence
    ids = tokenizer.encode(sequence,
        #TODO:Put something here,
        return_tensors='pt'
        )
    #TODO: Use the generate method in our model class to generate output tokens. 
    final_outputs = model.generate(ids,max_length = max_length, min_length = 3,
        #TODO: Complete some of the parameters and leave the parameters I've passed in.
        
        # Parameters you should leave
        do_sample=True,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    # Function to print and decode output.
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [41]:
#TODO: Now generate some text!
generate_text('I want to go', 16)

I want to go up in history at the end of my presidency and i think


In [68]:
generate_text('I want to go', 15)

loading configuration file ./result/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_size": 50257
}

loading weights file

I want to go out of our way to take action to protect the innocent


In [69]:
generate_text('I think that I might', 20)

loading configuration file ./result/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_size": 50257
}

loading weights file

I think that I might be allowed to apologize to my family or to the country if he was ever
